<h1>2b. Machine Learning using tf.estimator </h1>

In this notebook, we will create a machine learning model using tf.estimator and evaluate its performance.  The dataset is rather small (7700 samples), so we can do it all in-memory.  We will also simply pass the raw data in as-is. 

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import shutil

print(tf.__version__)

/usr/local/envs/py3env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.8.0


Read data created in the previous chapter.

In [2]:
# In CSV, label is the first column, after the features, followed by the key
CSV_COLUMNS = ['fare_amount', 'pickuplon','pickuplat','dropofflon','dropofflat','passengers', 'key']
FEATURES = CSV_COLUMNS[1:len(CSV_COLUMNS) - 1]
LABEL = CSV_COLUMNS[0]

df_train = pd.read_csv('./taxi-train.csv', header = None, names = CSV_COLUMNS)
df_valid = pd.read_csv('./taxi-valid.csv', header = None, names = CSV_COLUMNS)
df_test = pd.read_csv('./taxi-test.csv', header = None, names = CSV_COLUMNS)

In [3]:
df_train.head()

,fare_amount,pickuplon,pickuplat,dropofflon,dropofflat,passengers,key
0,12.0,-73.987625,40.750617,-73.971163,40.785180,1,0
1,4.5,-73.963620,40.774363,-73.953485,40.772665,1,1
2,4.5,-73.989649,40.756633,-73.985597,40.765662,1,2
3,10.0,-73.993950,40.727524,-74.006584,40.744240,1,3
4,2.5,-73.950223,40.668960,-73.948112,40.668872,6,4


<h2> Train and eval input functions to read from Pandas Dataframe </h2>

In [4]:
def make_train_input_fn(df, num_epochs):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = df[LABEL],
    batch_size = 128,
    num_epochs = num_epochs,
    shuffle = True,
    queue_capacity = 1000
  )

In [5]:
def make_eval_input_fn(df):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = df[LABEL],
    batch_size = 128,
    shuffle = False,
    queue_capacity = 1000
  )

Our input function for predictions is the same except we don't provide a label

In [6]:
def make_prediction_input_fn(df):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = None,
    batch_size = 128,
    shuffle = False,
    queue_capacity = 1000
  )

### Create feature columns for estimator

In [7]:
def make_feature_cols():
  input_columns = [tf.feature_column.numeric_column(k) for k in FEATURES]
  return input_columns

In [8]:
make_feature_cols()

[_NumericColumn(key='pickuplon', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='pickuplat', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='dropofflon', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='dropofflat', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='passengers', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

<h3> Linear Regression with tf.Estimator framework </h3>

In [9]:
tf.logging.set_verbosity(tf.logging.INFO)

OUTDIR = 'taxi_trained'
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time

model = tf.estimator.LinearRegressor(
      feature_columns = make_feature_cols(), model_dir = OUTDIR)

model.train(input_fn = make_train_input_fn(df_train, num_epochs = 10))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_evaluation_master': '', '_global_id_in_cluster': 0, '_task_type': 'worker', '_task_id': 0, '_model_dir': 'taxi_trained', '_is_chief': True, '_save_summary_steps': 100, '_train_distribute': None, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f6ec1fde198>, '_num_worker_replicas': 1, '_session_config': None, '_num_ps_replicas': 0, '_save_checkpoints_secs': 600, '_tf_random_seed': None, '_master': '', '_save_checkpoints_steps': None}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into taxi_trained/model.ckpt.
INFO:tensorflow:loss = 32435.098, st

Evaluate on the validation data (we should defer using the test data to after we have selected a final model).

In [11]:
def print_rmse(model, df):
  metrics = model.evaluate(input_fn = make_eval_input_fn(df))
  print('RMSE on dataset = {}'.format(np.sqrt(metrics['average_loss'])))

print_rmse(model, df_valid)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-02-19-17:02:01
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from taxi_trained/model.ckpt-608
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-02-19-17:02:02
INFO:tensorflow:Saving dict for global step 608: average_loss = 111.737724, global_step = 608, loss = 13288.808
RMSE on dataset = 10.570606231689453


This is nowhere near our benchmark (RMSE of $6 or so on this data), but it serves to demonstrate what TensorFlow code looks like.  Let's use this model for prediction.

In [12]:
predictions = model.predict(input_fn = make_prediction_input_fn(df_test))
for items in predictions:
  print(items)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from taxi_trained/model.ckpt-608
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'predictions': array([9.920511], dtype=float32)}
{'predictions': array([9.918189], dtype=float32)}
{'predictions': array([9.918973], dtype=float32)}
{'predictions': array([9.916745], dtype=float32)}
{'predictions': array([9.920272], dtype=float32)}
{'predictions': array([9.9200735], dtype=float32)}
{'predictions': array([9.918778], dtype=float32)}
{'predictions': array([9.918807], dtype=float32)}
{'predictions': array([9.920411], dtype=float32)}
{'predictions': array([9.918461], dtype=float32)}
{'predictions': array([9.920512], dtype=float32)}
{'predictions': array([9.920663], dtype=float32)}
{'predictions': array([9.914538], dtype=float32)}
{'predictions': array([9.918109], dtype=float32)}
{'predictions': array([9.974132], dt

{'predictions': array([9.919339], dtype=float32)}
{'predictions': array([10.140969], dtype=float32)}
{'predictions': array([9.917774], dtype=float32)}
{'predictions': array([9.91978], dtype=float32)}
{'predictions': array([9.973964], dtype=float32)}
{'predictions': array([9.974649], dtype=float32)}
{'predictions': array([10.0281725], dtype=float32)}
{'predictions': array([9.918824], dtype=float32)}
{'predictions': array([10.140452], dtype=float32)}
{'predictions': array([9.917938], dtype=float32)}
{'predictions': array([10.0287695], dtype=float32)}
{'predictions': array([9.91401], dtype=float32)}
{'predictions': array([9.911644], dtype=float32)}
{'predictions': array([9.91664], dtype=float32)}
{'predictions': array([9.91814], dtype=float32)}
{'predictions': array([9.902337], dtype=float32)}
{'predictions': array([9.918384], dtype=float32)}
{'predictions': array([9.905342], dtype=float32)}
{'predictions': array([9.973449], dtype=float32)}
{'predictions': array([9.920379], dtype=float32)

{'predictions': array([9.919033], dtype=float32)}
{'predictions': array([9.91953], dtype=float32)}
{'predictions': array([9.918903], dtype=float32)}
{'predictions': array([9.918635], dtype=float32)}
{'predictions': array([9.919169], dtype=float32)}
{'predictions': array([9.918836], dtype=float32)}
{'predictions': array([9.918877], dtype=float32)}
{'predictions': array([10.137644], dtype=float32)}
{'predictions': array([9.900739], dtype=float32)}
{'predictions': array([9.921702], dtype=float32)}
{'predictions': array([9.917888], dtype=float32)}
{'predictions': array([9.973737], dtype=float32)}
{'predictions': array([9.973914], dtype=float32)}
{'predictions': array([9.91948], dtype=float32)}
{'predictions': array([9.918053], dtype=float32)}
{'predictions': array([9.91812], dtype=float32)}
{'predictions': array([9.919359], dtype=float32)}
{'predictions': array([9.904933], dtype=float32)}
{'predictions': array([9.916768], dtype=float32)}
{'predictions': array([10.028366], dtype=float32)}
{

{'predictions': array([9.97252], dtype=float32)}
{'predictions': array([10.138902], dtype=float32)}
{'predictions': array([10.028054], dtype=float32)}
{'predictions': array([9.919045], dtype=float32)}
{'predictions': array([9.919535], dtype=float32)}
{'predictions': array([9.915792], dtype=float32)}
{'predictions': array([9.917836], dtype=float32)}
{'predictions': array([9.972354], dtype=float32)}
{'predictions': array([9.969454], dtype=float32)}
{'predictions': array([9.918738], dtype=float32)}
{'predictions': array([10.195653], dtype=float32)}
{'predictions': array([9.918501], dtype=float32)}
{'predictions': array([10.197068], dtype=float32)}
{'predictions': array([9.972308], dtype=float32)}
{'predictions': array([9.918015], dtype=float32)}
{'predictions': array([9.9182005], dtype=float32)}
{'predictions': array([9.97264], dtype=float32)}
{'predictions': array([9.906124], dtype=float32)}
{'predictions': array([9.917426], dtype=float32)}
{'predictions': array([9.972714], dtype=float32

{'predictions': array([9.918372], dtype=float32)}
{'predictions': array([10.132152], dtype=float32)}
{'predictions': array([10.14038], dtype=float32)}
{'predictions': array([9.973347], dtype=float32)}
{'predictions': array([9.915135], dtype=float32)}
{'predictions': array([9.918788], dtype=float32)}
{'predictions': array([9.919452], dtype=float32)}
{'predictions': array([9.97054], dtype=float32)}
{'predictions': array([9.972682], dtype=float32)}
{'predictions': array([10.137588], dtype=float32)}
{'predictions': array([9.918996], dtype=float32)}
{'predictions': array([10.029219], dtype=float32)}
{'predictions': array([9.918427], dtype=float32)}
{'predictions': array([9.922177], dtype=float32)}
{'predictions': array([9.97465], dtype=float32)}
{'predictions': array([9.9756155], dtype=float32)}
{'predictions': array([9.917423], dtype=float32)}
{'predictions': array([9.919097], dtype=float32)}
{'predictions': array([9.919718], dtype=float32)}
{'predictions': array([9.960529], dtype=float32)

{'predictions': array([9.917086], dtype=float32)}
{'predictions': array([9.919064], dtype=float32)}
{'predictions': array([9.973], dtype=float32)}
{'predictions': array([9.919492], dtype=float32)}
{'predictions': array([9.964579], dtype=float32)}
{'predictions': array([9.974153], dtype=float32)}
{'predictions': array([9.918457], dtype=float32)}
{'predictions': array([9.974068], dtype=float32)}
{'predictions': array([9.920161], dtype=float32)}
{'predictions': array([9.974798], dtype=float32)}
{'predictions': array([9.918656], dtype=float32)}
{'predictions': array([9.919735], dtype=float32)}
{'predictions': array([9.974921], dtype=float32)}
{'predictions': array([9.91712], dtype=float32)}
{'predictions': array([10.03036], dtype=float32)}
{'predictions': array([9.918593], dtype=float32)}
{'predictions': array([9.969768], dtype=float32)}
{'predictions': array([10.194455], dtype=float32)}
{'predictions': array([10.02949], dtype=float32)}
{'predictions': array([10.029688], dtype=float32)}
{'

This explains why the RMSE was so high -- the model essentially predicts the same amount for every trip.  Would a more complex model help? Let's try using a deep neural network.  The code to do this is quite straightforward as well.

<h3> Deep Neural Network regression </h3>

In [13]:
tf.logging.set_verbosity(tf.logging.INFO)
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time

model = tf.estimator.DNNRegressor(hidden_units = [32, 8, 2], feature_columns = make_feature_cols(), model_dir = OUTDIR)
model.train(input_fn = make_train_input_fn(df_train, num_epochs = 100));
print_rmse(model, df_valid)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_evaluation_master': '', '_global_id_in_cluster': 0, '_task_type': 'worker', '_task_id': 0, '_model_dir': 'taxi_trained', '_is_chief': True, '_save_summary_steps': 100, '_train_distribute': None, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f6e9d390828>, '_num_worker_replicas': 1, '_session_config': None, '_num_ps_replicas': 0, '_save_checkpoints_secs': 600, '_tf_random_seed': None, '_master': '', '_save_checkpoints_steps': None}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into taxi_trained/model.ckpt.
INFO:tensorflow:loss = 26131.998, st

We are not beating our benchmark with either model ... what's up?  Well, we may be using TensorFlow for Machine Learning, but we are not yet using it well.  That's what the rest of this course is about!

But, for the record, let's say we had to choose between the two models. We'd choose the one with the lower validation error. Finally, we'd measure the RMSE on the test data with this chosen model.

<h2> Benchmark dataset </h2>

Let's do this on the benchmark dataset.

In [14]:
import google.datalab.bigquery as bq
import numpy as np
import pandas as pd

def create_query(phase, EVERY_N):
  """
  phase: 1 = train 2 = valid
  """
  base_query = """
SELECT
  (tolls_amount + fare_amount) AS fare_amount,
  EXTRACT(DAYOFWEEK FROM pickup_datetime) * 1.0 AS dayofweek,
  EXTRACT(HOUR FROM pickup_datetime) * 1.0 AS hourofday,
  pickup_longitude AS pickuplon,
  pickup_latitude AS pickuplat,
  dropoff_longitude AS dropofflon,
  dropoff_latitude AS dropofflat,
  passenger_count*1.0 AS passengers,
  CONCAT(CAST(pickup_datetime AS STRING), CAST(pickup_longitude AS STRING), CAST(pickup_latitude AS STRING), CAST(dropoff_latitude AS STRING), CAST(dropoff_longitude AS STRING)) AS key
FROM
  `nyc-tlc.yellow.trips`
WHERE
  trip_distance > 0
  AND fare_amount >= 2.5
  AND pickup_longitude > -78
  AND pickup_longitude < -70
  AND dropoff_longitude > -78
  AND dropoff_longitude < -70
  AND pickup_latitude > 37
  AND pickup_latitude < 45
  AND dropoff_latitude > 37
  AND dropoff_latitude < 45
  AND passenger_count > 0
  """

  if EVERY_N == None:
    if phase < 2:
      # Training
      query = "{0} AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), 4) < 2".format(base_query)
    else:
      # Validation
      query = "{0} AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), 4) = {1}".format(base_query, phase)
  else:
    query = "{0} AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), {1}) = {2}".format(base_query, EVERY_N, phase)
    
  return query

query = create_query(2, 100000)
df = bq.Query(query).execute().result().to_dataframe()

In [15]:
print_rmse(model, df)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-02-19-17:08:53
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from taxi_trained/model.ckpt-6071
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-02-19-17:08:53
INFO:tensorflow:Saving dict for global step 6071: average_loss = 110.68731, global_step = 6071, loss = 14063.724
RMSE on dataset = 10.520803451538086


RMSE on benchmark dataset is <b>9.41</b> (your results will vary because of random seeds).

This is not only way more than our original benchmark of 6.00, but it doesn't even beat our distance-based rule's RMSE of 8.02.

Fear not -- you have learned how to write a TensorFlow model, but not to do all the things that you will have to do to your ML model performant. We will do this in the next chapters. In this chapter though, we will get our TensorFlow model ready for these improvements.

In a software sense, the rest of the labs in this chapter will be about refactoring the code so that we can improve it.

## Challenge Exercise

Create a neural network that is capable of finding the volume of a cylinder given the radius of its base (r) and its height (h). Assume that the radius and height of the cylinder are both in the range 0.5 to 2.0. Simulate the necessary training dataset.
<p>
Hint (highlight to see):
<p style='color:white'>
The input features will be r and h and the label will be $\pi r^2 h$
Create random values for r and h and compute V.
Your dataset will consist of r, h and V.
Then, use a DNN regressor.
Make sure to generate enough data.
</p>

Copyright 2017 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License